In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LSTM + Attention Experiment

In [26]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch_geometric.data import Batch

In [27]:
train_npz = np.load('../../train.npz')
train_data = train_npz['data']
test_npz  = np.load('../../test_input.npz')
test_data  = test_npz['data']

In [28]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [29]:
import sys
sys.path.append('..')
from TrajectoryDataset import TrajectoryDatasetTrain, TrajectoryDatasetTest
from utils import train_model

sys.path.append('../..')
from models.lstm_attention import LSTMAttentionModel

In [30]:
torch.manual_seed(251)
np.random.seed(42)

scale = 5.0

N = len(train_data)
val_size = int(0.05 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


#### Your Optimizer and Hyperparameters for Question 2 Problem A (Try to use different optimizers and hyperparameters for your model and see how it affects the performance of your model)

In [31]:
model = LSTMAttentionModel()

optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=4)
early_stopping_patience = 30 
criterion = nn.MSELoss()

In [ ]:
model = model.to(device)
train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    device, 
    optimizer, 
    criterion, 
    scheduler, 
    early_stopping_patience,
    "lstm_attention_weights"
)

Epoch:   1%|          | 1/100 [00:28<47:30, 28.79s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE   3.5416 | val normalized MSE   1.2108, | val unnormalized MAE   3.3539 | val unnormalized MSE  30.2694


Epoch:   2%|▏         | 2/100 [00:59<48:24, 29.63s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE   0.8907 | val normalized MSE   0.8217, | val unnormalized MAE   2.5884 | val unnormalized MSE  20.5415


Epoch:   3%|▎         | 3/100 [01:27<47:09, 29.16s/epoch]

...

Epoch 097 | Learning rate 0.000237 | train normalized MSE   0.2125 | val normalized MSE   0.2895, | val unnormalized MAE   1.2935 | val unnormalized MSE   7.2382


Epoch:  99%|█████████▉| 99/100 [1:05:36<00:27, 27.26s/epoch]

Epoch 098 | Learning rate 0.000237 | train normalized MSE   0.2109 | val normalized MSE   0.2826, | val unnormalized MAE   1.2690 | val unnormalized MSE   7.0649


Epoch: 100%|██████████| 100/100 [1:06:04<00:00, 39.65s/epoch]

Epoch 099 | Learning rate 0.000237 | train normalized MSE   0.2130 | val normalized MSE   0.2930, | val unnormalized MAE   1.2992 | val unnormalized MSE   7.3255


In [33]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

lstm_attention_weights = torch.load("lstm_attention_weights.pt")
model.load_state_dict(lstm_attention_weights)
torch.save(model, "full_lstm_attention_model.pt")

test_dataset.generate_submission_predictions(model, device, output_file_name='lstm_attention_submission')